Stajesz przed nowym zadaniem, nowy product manager poprosił Cię o pomoc w obliczeniu i wizualizacji nowego KPI, który ma
pomóc ulepszyć proces dostarczania rezultatów do klienta. Jego definicja to: Ilość zaakceptowanych błędów na godzinę w 
pierwszych 6-ciu godzinach od rozpoczęcia cyklu testowego. PM przekazał Ci, że dla uproszczenia jako początek cyklu testowego możesz uznać czas pierwszego zgłoszonego błędu. 

Jeżeli uda Ci się pokazać tą metrykę, PM napewno doceni analizę na tych danych - ponieważ on nie ma na to czasu.

Stwórz widok pokazujący powyższą metryke i zaproponuj dodatkowe wykresy z które mogą być przydatne dla Product Manager'a.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

In [81]:
data = pd.read_csv('cycle_data.csv')

In [3]:
data.sample(10)

,issue_code_,created_at,severity,resolution,affected_components,creator_id
68,CYCLE-10-69,13/04/2021 17:04:33,High,Confirmed (S),{Content},1120.0
29,CYCLE-10-30,13/04/2021 16:05:45,Low,Confirmed (S),{Workouts},8124.0
69,CYCLE-10-70,13/04/2021 17:07:06,Low,Confirmed (S),{Workouts},5271.0
26,CYCLE-10-27,13/04/2021 16:03:13,High,Confirmed (S),{Profile},4075.0
19,CYCLE-10-20,13/04/2021 15:58:32,Medium,Confirmed (S),{Profile},3230.0
6,CYCLE-10-7,13/04/2021 15:46:35,Low,Confirmed (S),{Signup},4693.0
7,CYCLE-10-8,13/04/2021 15:46:44,Medium,Confirmed (S),Profile,8058.0
32,CYCLE-10-33,13/04/2021 16:09:21,High,Confirmed (S),{Content},4693.0
66,CYCLE-10-67,13/04/2021 17:02:25,High,Confirmed (S),{Meals},8124.0
38,CYCLE-10-39,13/04/2021 16:17:27,Medium,Confirmed (S),{Workouts},4075.0


In [4]:
data.severity.unique()

array(['Low', 'High', 'Medium', 'Critical'], dtype=object)

In [5]:
data.resolution.unique()

array(['Confirmed (S)', 'Duplicate', 'Expected Behaviour',
       'Confirmed (!)', 'Invalid - not tester error', 'Not in Scope',
       'No tester response', 'Confirm (S)'], dtype=object)

In [6]:
data.affected_components.unique()

array(['{Signup}', '{Profile}', 'Profile', '{Sign-up}', '{Content}',
       '{Challenge}', '{Workouts}', '{Settings}', '{Kit}', '{Guides}',
       '{Meals}', '{Login}'], dtype=object)

In [82]:
# First step will be to clear the data.
# I replace rows where resolution status is 'Confirmed (!)' and 'Confirm (S)' with status 'Confirmed (S)', because it's 
# the same category 

data["resolution"].replace({"Confirmed (!)": "Confirmed (S)", "Confirm (S)": "Confirmed (S)"}, inplace=True)
data.resolution.unique()

array(['Confirmed (S)', 'Duplicate', 'Expected Behaviour',
       'Invalid - not tester error', 'Not in Scope', 'No tester response'],
      dtype=object)

In [8]:
data.isnull().sum()

issue_code_            0
created_at             1
severity               0
resolution             0
affected_components    0
creator_id             1
dtype: int64

In [83]:
# Drop rows where NaN is in 'created_at' column, because we need this data to check if error was created within first 
# 6 hours of testing cycle.

data.dropna(subset=['created_at'], inplace=True)
data['created_at'].isnull().sum()

0

In [10]:
data.head(10)

,issue_code_,created_at,severity,resolution,affected_components,creator_id
0,CYCLE-10-1,13/04/2021 15:35:36,Low,Confirmed (S),{Signup},8124.0
1,CYCLE-10-2,13/04/2021 15:40:15,Low,Confirmed (S),{Signup},8124.0
2,CYCLE-10-3,13/04/2021 15:41:28,Low,Confirmed (S),{Signup},1066.0
3,CYCLE-10-4,13/04/2021 15:41:38,High,Confirmed (S),{Signup},4075.0
4,CYCLE-10-5,13/04/2021 15:43:07,Medium,Confirmed (S),{Profile},3230.0
5,CYCLE-10-6,13/04/2021 15:43:32,Low,Confirmed (S),{Signup},5271.0
6,CYCLE-10-7,13/04/2021 15:46:35,Low,Confirmed (S),{Signup},4693.0
7,CYCLE-10-8,13/04/2021 15:46:44,Medium,Confirmed (S),Profile,8058.0
8,CYCLE-10-9,13/04/2021 15:46:53,High,Confirmed (S),{Profile},3230.0
9,CYCLE-10-10,13/04/2021 15:47:03,Low,Duplicate,{Signup},1066.0


In [11]:
type(data['created_at'][0])

str

In [84]:
# I want to change data type in column "created_at" to datetime 
# There is an error appearing because of wrong format in one row, so I'm going to change this

data[data['created_at'] == '04/13/2021 16:25:40']

,issue_code_,created_at,severity,resolution,affected_components,creator_id
44,CYCLE-10-45,04/13/2021 16:25:40,High,Confirmed (S),{Workouts},4693.0


In [85]:
data.at[44, 'created_at'] = '13/04/2021 16:25:40'
data.loc[44, 'created_at']

'13/04/2021 16:25:40'

In [86]:
# Change format 
data['created_at'] = pd.to_datetime(data['created_at'], format='%d/%m/%Y %H:%M:%S')

In [15]:
type(data['created_at'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [16]:
data.head()

,issue_code_,created_at,severity,resolution,affected_components,creator_id
0,CYCLE-10-1,2021-04-13 15:35:36,Low,Confirmed (S),{Signup},8124.0
1,CYCLE-10-2,2021-04-13 15:40:15,Low,Confirmed (S),{Signup},8124.0
2,CYCLE-10-3,2021-04-13 15:41:28,Low,Confirmed (S),{Signup},1066.0
3,CYCLE-10-4,2021-04-13 15:41:38,High,Confirmed (S),{Signup},4075.0
4,CYCLE-10-5,2021-04-13 15:43:07,Medium,Confirmed (S),{Profile},3230.0


In [87]:
# Now I'm going to create a new column 'cycle_hour' with information about hour of testing cycle (from 1 to 6) in which error appeared
start_date = data.loc[0, 'created_at']
end_date = start_date + pd.DateOffset(hours=1)
start_date, end_date

(Timestamp('2021-04-13 15:35:36'), Timestamp('2021-04-13 16:35:36'))

In [88]:
for i in range(6):
    data.loc[(data['created_at'] >= start_date + pd.DateOffset(hours=i)) & (data['created_at'] < end_date + pd.DateOffset(hours=i)), 'cycle_hour'] = i+1

In [89]:
# Drop rows if issue appeared after 6 hours.
data.dropna(subset=['cycle_hour'], inplace=True)
data['cycle_hour'].isnull().sum()

0

In [90]:
data['cycle_hour'].unique()

array([1., 2., 3., 4., 5., 6.])

In [104]:
confirmed_data = data[data['resolution'] == 'Confirmed (S)']

In [105]:
confirmed_data['cycle_hour'].unique()

array([1., 2., 3., 4., 5., 6.])

In [116]:
confirmed_by_hours = (
    confirmed_data
    .groupby(['cycle_hour'])
    ['issue_code_'].count()
    .reset_index()
    .rename(columns={'issue_code_': 'number of issues'})
    
)

confirmed_by_hours

,cycle_hour,number of issues
0,1.0,47
1,2.0,16
2,3.0,4
3,4.0,5
4,5.0,2
5,6.0,5


In [ ]:
# Plot o

In [115]:
confirmed_by_hours_severity = (
    confirmed_data
    .groupby(['cycle_hour', 'severity'])
    ['issue_code_'].count()
    .reset_index()
    .rename(columns={'issue_code_': 'number of issues'})
    
)

confirmed_by_hours_severity

,cycle_hour,severity,number of issues
0,1.0,Critical,3
1,1.0,High,12
2,1.0,Low,17
3,1.0,Medium,15
4,2.0,Critical,1
5,2.0,High,5
6,2.0,Low,3
7,2.0,Medium,7
8,3.0,High,2
9,3.0,Low,1
